In [1]:
import aocd
import re
from dotenv import load_dotenv
import numpy as np
from tqdm import tqdm
import networkx as nx

load_dotenv()

puzzle = aocd.get_puzzle(day=15, year=2024)
text = puzzle.input_data
examples = puzzle.examples
print(examples)

lines = text.splitlines()

example = examples[0].input_data.splitlines()

example


[Example(input_data='##########\n#..O..O.O#\n#......O.#\n#.OO..O.O#\n#..O@..O.#\n#O#..O...#\n#O..O..O.#\n#.OO.O.OO#\n#....O...#\n##########\n\n<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^\nvvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v\n><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<\n<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^\n^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><\n^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^\n>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^\n<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>\n^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>\nv^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^', answer_a='2028', answer_b=None, extra=None)]


['##########',
 '#..O..O.O#',
 '#......O.#',
 '#.OO..O.O#',
 '#..O@..O.#',
 '#O#..O...#',
 '#O..O..O.#',
 '#.OO.O.OO#',
 '#....O...#',
 '##########',
 '',
 '<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^',
 'vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v',
 '><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<',
 '<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^',
 '^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><',
 '^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^',
 '>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^',
 '<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>',
 '^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>',
 'v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^']

In [9]:
lines = example
text = examples[0].input_data

In [2]:
import re
import numpy as np

ps = np.array([tuple(map(int, ll.split()[0].removeprefix("p=").split(","))) for ll in lines])
vs = np.array([tuple(map(int, ll.split()[1].removeprefix("v=").split(","))) for ll in lines])

# shape = np.array([11,7])
shape = np.array([101,103])

end = (ps + 100*vs)%shape

q1 = (end < shape//2).all(axis=1).sum()
q2 = ((end[:,0] < shape[0]//2) & (end[:,1] > shape[1]//2)).sum()
q3 = ((end[:,0] > shape[0]//2) & (end[:,1] < shape[1]//2)).sum()
q4 = (end > shape//2).all(axis=1).sum()

s = q1*q2*q3*q4

print(q1*q2*q3*q4)


222901875


In [9]:
es = [((ps + i*vs)%shape) for i in range(1,15000)]

es = np.array(es)
print(es.shape)

(es == es[0]).all(axis=2).all(axis=1).sum()



(14999, 500, 2)


np.int64(2)

In [10]:
aocd.submit(s, day=14, year=2024)

answer a: 222901875
submitting for part b (part a is already completed)
coerced int64 value np.int64(222901875) for 2024/14 to '222901875'


AocdError: cowardly refusing to submit 222901875 for part b, because that was the answer for part a

In [55]:
from tqdm import trange
imgs = np.zeros((15000, *shape))
print(imgs.shape)

for i in trange(15000):
    end = (ps + i*vs)%shape
    z = np.zeros(shape)
    for p in end:
        p1,p2 = p
        z[p1,p2] = 1
    imgs[i] = z

    

imgs.shape

(15000, 101, 103)


100%|██████████| 15000/15000 [00:14<00:00, 1035.90it/s]


(15000, 101, 103)

In [ ]:
imgs.sum(2).sum(1).argmax()



np.int64(6243)

In [32]:

def l_connected_comp(e):
    g = nx.Graph()
    for ee in e:
        g.add_node(tuple(ee))

    for ee in e:
        for gg in e:
            if abs(ee[0]-gg[0]) + abs(ee[1]-gg[1]) < 2:
                g.add_edge(tuple(ee),tuple(gg))
    
    return max([len(c) for c in nx.connected_components(g)])

ls = []
for i in range(3000,6000):
    ls.append((l_connected_comp((ps + i*vs)%shape), i))
    if i % 100 == 0:
        print(i)

print(max(ls))



3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
(12, 3314)


In [61]:
e = (ps + 6243*vs)%shape

print(len(e))

z = np.zeros(shape)
for p in e:
    p1,p2 = p
    z[p1,p2] += 1

def s(i):
    if i == 0:
        return "."
    return str(int(i))

for row in z:
    print("".join([s(ss) for ss in row]))

500
..........................................................................................1............
.....1...................................................1...........................................1.
.........................................................1...........................1.................
.................................................................................................1.....
.......................................................................................................
...............................................................1.........................1...........1.
...................................................................1..................1................
...........................................................1...........................................
...............................................................1.......................................
.............1..........................................1...

In [ ]:
aocd.submit(6243, day=14)

answer a: 222901875
submitting for part b (part a is already completed)


In [54]:
# save as jpgs
from PIL import Image


for i in trange(15000):
    img = imgs[i]
    img = (img == 0) * 255
    img = img.astype(np.uint8)
    img = Image.fromarray(img)
    img.save(f"imgs/{i:06d}.jpg")

100%|██████████| 15000/15000 [00:03<00:00, 3831.70it/s]


In [38]:
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 2.7 MB/s eta 0:00:0000:0100:01


In [53]:
import cv2

def edge_density(image):
    """
    Detect edges using Canny filter and calculate edge density.
    Edge density is the proportion of pixels classified as edges.
    """
    i = ((image > 0)*256).astype(np.uint8)

    # Apply Canny edge detection
    edges = cv2.Canny(i, 10,20)  # thresholds can be adjusted
    
    # Calculate the proportion of edge pixels
    edge_pixels = np.sum(edges > 0)
    total_pixels = image.shape[0] * image.shape[1]
    
    return edge_pixels / total_pixels

es = [edge_density(i) for i in imgs]

max([(edge_density(img), i) for i,img in enumerate(imgs)])


(np.float64(0.0), 14999)